In [10]:
%%writefile "backtest_1.py"
import pandas as pd
import psycopg2 as psql

from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()



if __name__ == '__main__':
    connection = psql.connect(database='okx', user='okx', password='example',
                              host='127.0.0.1', port=5432)
    RENAME_COLS = {'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}
    df = pd.read_sql("select * from okx_candle_btc_usdt_1m;", connection)\
            .rename(columns=RENAME_COLS)
    df[['Open', 'High', 'Low', 'Close', 'Volume']] = df[['Open', 'High', 'Low', 'Close', 'Volume']] / 1e6
    df.ts = pd.to_datetime(df.ts)
    print(df.head)
    

    bt = Backtest(df, SmaCross, cash=10_000, commission=.002)
    stats = bt.run()
    print(stats)


Writing backtest_1.py


In [9]:
stats

Start                                     0.0
End                                   53993.0
Duration                              53993.0
Exposure Time [%]                    98.68874
Equity Final [$]                      0.11694
Equity Peak [$]                       10000.0
Commissions [$]                   10069.74612
Return [%]                          -99.99883
Buy & Hold Return [%]                 1.20733
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Alpha [%]                           -99.93276
Beta                                 -0.05472
Max. Drawdown [%]                   -99.99883
Avg. Drawdown [%]                   -99.99883
Max. Drawdown Duration                53935.0
Avg. Drawdown Duration                53935.0
# Trades                               2957.0
Win Rate [%]                      